In [ ]:
!pip install transformers


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
fake = pd.read_csv("/content/drive/MyDrive/NLP PROJECT/DATASET/Fake.csv")
true = pd.read_csv("/content/drive/MyDrive/NLP PROJECT/DATASET/True.csv")
# Add category labels
fake['category'] = 1
true['category'] = 0


In [ ]:
# Concatenate datasets and reset index
df = pd.concat([fake, true]).reset_index(drop=True)

In [ ]:
# Separate the majority and minority classes
df_majority = df[df['category'] == 1]
df_minority = df[df['category'] == 0]

# Sample 3000 data points from each class
df_majority_sampled = df_majority.sample(n=500, random_state=42)
df_minority_sampled = df_minority.sample(n=500, random_state=42)

# Combine the sampled data
df_balanced = pd.concat([df_majority_sampled, df_minority_sampled])

# Shuffle the dataframe
df= df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the balanced dataset if needed
# df_balanced.to_csv('balanced_dataset.csv', index=False)

df_majority = df[df['category'] == 1]
df_minority = df[df['category'] == 0]

# Undersample the majority class
df_majority_undersampled = df_majority.sample(len(df_minority), random_state=42)

# Combine the undersampled majority class with the minority class
df = pd.concat([df_majority_undersampled, df_minority])

In [ ]:
df=df[['text','category']]

In [ ]:
batch_1 = df[:1000]

In [ ]:
batch_1['category'].value_counts()

category
1    500
0    500
Name: count, dtype: int64

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

In [ ]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
# Tokenize the data with truncation and padding to max length
encoded_inputs = tokenizer.batch_encode_plus(
    df['text'].tolist(),
    add_special_tokens=True,
    max_length=512,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

In [ ]:
# Extract input_ids and attention_mask
input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']

In [ ]:
# Verify the shape of input_ids
print(input_ids.shape)  # Should print (1000, 512)

torch.Size([1000, 512])


In [ ]:
# Get the features from the model
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
!pip install transformers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

# Load datasets
fake = pd.read_csv("/content/drive/MyDrive/NLP PROJECT/DATASET/Fake.csv")
true = pd.read_csv("/content/drive/MyDrive/NLP PROJECT/DATASET/True.csv")

# Add category labels
fake['category'] = 1
true['category'] = 0

# Concatenate datasets and reset index
df = pd.concat([fake, true]).reset_index(drop=True)

# Sample 500 data points from each class for the example
df_majority_sampled = df[df['category'] == 1].sample(n=500, random_state=42)
df_minority_sampled = df[df['category'] == 0].sample(n=500, random_state=42)

# Combine the sampled data and shuffle the dataframe
df_balanced = pd.concat([df_majority_sampled, df_minority_sampled]).sample(frac=1, random_state=42).reset_index(drop=True)

# Prepare the balanced dataframe
df = df_balanced[['text', 'category']]

# Load the BERT model and tokenizer
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

# Function to get BERT embeddings
def get_bert_embeddings(text_list):
    encoded_inputs = tokenizer.batch_encode_plus(
        text_list,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    input_ids = encoded_inputs['input_ids']
    attention_mask = encoded_inputs['attention_mask']
    with torch.no_grad():
        last_hidden_states = model(input_ids, attention_mask=attention_mask)
    return last_hidden_states[0][:,0,:].numpy()  # Use the [CLS] token representation

# Process data in batches
batch_size = 50
embeddings = []

for i in range(0, len(df), batch_size):
    batch_texts = df['text'][i:i + batch_size].tolist()
    batch_embeddings = get_bert_embeddings(batch_texts)
    embeddings.append(batch_embeddings)

# Combine all batch embeddings
X = np.vstack(embeddings)
y = df['category'].values

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)

# Create and train the Logistic Regression model
model_pipeline_lr = Pipeline([
    ('lr', LogisticRegression(max_iter=1000, random_state=1))
])
model_pipeline_lr.fit(X_train, y_train)

# Predict on the test set
y_pred_lr = model_pipeline_lr.predict(X_test)

# Evaluate the model
accuracy_lr = accuracy_score(y_test, y_pred_lr)
precision_lr = precision_score(y_test, y_pred_lr, average='weighted')
recall_lr = recall_score(y_test, y_pred_lr, average='weighted')
f1_score_lr = f1_score(y_test, y_pred_lr, average='weighted')
classification_report_lr = classification_report(y_test, y_pred_lr)

# Print evaluation metrics
print("Evaluation Metrics for Logistic Regression Model")
print("------------------------------------------------")
print(f"Accuracy: {accuracy_lr:.4f}")
print(f"Precision: {precision_lr:.4f}")
print(f"Recall: {recall_lr:.4f}")
print(f"F1-score: {f1_score_lr:.4f}")
print("Classification Report:")
print(classification_report_lr)


NameError: name 'Pipeline' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
